In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
df = pd.DataFrame(pd.read_excel('./datasci/cleanedup/whl_2025_base.xlsx'))
df.head()

,game_id,record_id,home_team,away_team,went_ot,home_off_line,home_def_pairing,away_off_line,away_def_pairing,home_goalie,...,home_goals,away_assists,away_shots,away_xg,away_max_xg,away_goals,home_penalties_committed,home_penalty_minutes,away_penalties_committed,away_penalty_minutes
0,game_0001,record_1,thailand,pakistan,0,PP_kill_dwn,PP_kill_dwn,PP_up,PP_up,player_id_142,...,0,2,9,1.4645,0.2166,1,7,14,1,2
1,game_0001,record_2,thailand,pakistan,0,second_off,second_def,second_off,second_def,player_id_142,...,0,2,1,0.0928,0.0928,1,0,0,0,0
2,game_0001,record_19,thailand,pakistan,0,second_off,first_def,first_off,second_def,player_id_142,...,0,0,0,0.0000,0.0000,0,0,0,0,0
3,game_0001,record_11,thailand,pakistan,0,second_off,first_def,second_off,second_def,player_id_142,...,0,0,1,0.0763,0.0763,0,0,0,0,0
4,game_0001,record_10,thailand,pakistan,0,second_off,second_def,first_off,first_def,player_id_142,...,0,0,0,0.0000,0.0000,0,0,0,0,0


In [ ]:
'''
IDENTIFIERS:
- game_id
- record_id

ENTITIES:
- home_team
- away_team
- home_goalie
- away_goalie

CONTEXT:
- home_off_line
- away_off_line
- home_def_pairing
- away_def_pairing
- went_ot

OUTCOMES:
- home_goals
- away_goals
- home_shots
- away_shots
- home_penalties_committed
- away_penalties_committed

DERIVED METRICS:
- home_xg
- away_xg
- home_max_xg
- away_max_xg
'''

In [3]:
sum_cols = [
    "home_goals", "away_goals",
    "home_shots", "away_shots",
    "home_xg", "away_xg",
    "home_assists", "away_assists",
    "home_penalties_committed", "away_penalties_committed",
    "home_penalty_minutes", "away_penalty_minutes"
]
first_cols = [
    "home_team",
    "away_team",
    "went_ot"
]
agg_dict = {}

for col in sum_cols:
    agg_dict[col] = "sum"

for col in first_cols:
    agg_dict[col] = "first"
games = (
    df
    .groupby("game_id", as_index=False)
    .agg(agg_dict)
)
games.shape

(1312, 16)

In [5]:
games["home_score"] = games["home_goals"]
games["away_score"] = games["away_goals"]

games["goal_diff"] = games["home_score"] - games["away_score"]
games["total_goals"] = games["home_score"] + games["away_score"]

games["shot_diff"] = games["home_shots"] - games["away_shots"]
games["total_shots"] = games["home_shots"] + games["away_shots"]

games


,game_id,home_goals,away_goals,home_shots,away_shots,home_xg,away_xg,home_assists,away_assists,home_penalties_committed,...,away_penalty_minutes,home_team,away_team,went_ot,home_score,away_score,goal_diff,total_goals,shot_diff,total_shots
0,game_0001,1,3,21,24,2.8231,2.7516,2,6,8,...,12,thailand,pakistan,0,1,3,-2,4,-3,45
1,game_0002,3,2,31,27,3.4075,3.0509,4,4,9,...,12,france,morocco,1,3,2,1,5,4,58
2,game_0003,4,3,42,26,4.8049,2.3901,6,6,6,...,24,peru,iceland,1,4,3,1,7,16,68
3,game_0004,0,1,19,16,1.6322,1.6166,0,2,6,...,10,mongolia,saudi_arabia,0,0,1,-1,1,3,35
4,game_0005,4,1,32,42,3.1535,4.7456,7,1,9,...,12,usa,guatemala,0,4,1,3,5,-10,74
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1307,game_1308,2,0,13,24,1.2708,2.4745,3,0,7,...,10,switzerland,france,0,2,0,2,2,-11,37
1308,game_1309,5,4,29,29,4.2460,2.3279,9,5,2,...,24,netherlands,south_korea,0,5,4,1,9,0,58
1309,game_1310,1,3,21,19,3.0211,2.1164,1,5,6,...,8,uk,uae,0,1,3,-2,4,2,40
1310,game_1311,3,0,18,24,2.4804,1.9921,5,0,6,...,13,brazil,rwanda,0,3,0,3,3,-6,42


In [6]:
home_games = games.copy()

home_games["team"] = home_games["home_team"]
home_games["opponent"] = home_games["away_team"]

home_games["goals_for"] = home_games["home_score"]
home_games["goals_against"] = home_games["away_score"]

home_games["shots_for"] = home_games["home_shots"]
home_games["shots_against"] = home_games["away_shots"]

home_games["xg_for"] = home_games["home_xg"]
home_games["xg_against"] = home_games["away_xg"]

home_games["is_home"] = 1

In [7]:
away_games = games.copy()

away_games["team"] = away_games["away_team"]
away_games["opponent"] = away_games["home_team"]

away_games["goals_for"] = away_games["away_score"]
away_games["goals_against"] = away_games["home_score"]

away_games["shots_for"] = away_games["away_shots"]
away_games["shots_against"] = away_games["home_shots"]

away_games["xg_for"] = away_games["away_xg"]
away_games["xg_against"] = away_games["home_xg"]

away_games["is_home"] = 0

In [15]:
team_games = pd.concat([home_games, away_games], ignore_index=True)
team_games["goal_diff"] = team_games["goals_for"] - team_games["goals_against"]

team_games["win"] = (team_games["goal_diff"] > 0).astype(int)
team_games["loss"] = (team_games["goal_diff"] < 0).astype(int)
team_season = (
    team_games
    .groupby("team", as_index=False)
    .agg(
        games_played = ("team", "count"),
        wins = ("win", "sum"),
        losses = ("loss", "sum"),
        goals_for = ("goals_for", "sum"),
        goals_against = ("goals_against", "sum"),
        shots_for = ("shots_for", "sum"),
        shots_against = ("shots_against", "sum"),
        xg_for = ("xg_for", "sum"),
        xg_against = ("xg_against", "sum"),
        avg_goal_diff = ("goal_diff", "mean"),
        home_games = ("is_home", "sum")
    )
)
team_season["goals_per_game"] = team_season["goals_for"] / team_season["games_played"]
team_season["goals_against_per_game"] = team_season["goals_against"] / team_season["games_played"]

team_season["shot_diff"] = team_season["shots_for"] - team_season["shots_against"]
team_season["xg_diff"] = team_season["xg_for"] - team_season["xg_against"]
team_season["win_pct"] = team_season["wins"] / team_season["games_played"]
team_season["loss_pct"] = team_season["losses"] / team_season["games_played"]

team_season
team_season.sort_values("loss_pct", ascending=False).head()


,team,games_played,wins,losses,goals_for,goals_against,shots_for,shots_against,xg_for,xg_against,avg_goal_diff,home_games,goals_per_game,goals_against_per_game,shot_diff,xg_diff,win_pct,loss_pct
0,brazil,82,58,24,276,189,2162,2242,272.4793,221.8266,1.060976,41,3.365854,2.304878,-80,50.6527,0.707317,0.292683
14,netherlands,82,54,28,243,174,2124,2018,242.4986,201.8032,0.841463,41,2.963415,2.121951,106,40.6954,0.658537,0.341463
19,peru,82,52,30,256,178,2311,2001,244.5499,214.9336,0.951220,41,3.121951,2.170732,310,29.6163,0.634146,0.365854
27,thailand,82,50,32,294,248,2346,2127,294.7369,221.6617,0.560976,41,3.585366,3.024390,219,73.0752,0.609756,0.390244
8,india,82,49,33,221,193,2373,2208,239.3359,237.9323,0.341463,41,2.695122,2.353659,165,1.4036,0.597561,0.402439
17,pakistan,82,49,33,263,212,2457,1988,284.7034,233.4620,0.621951,41,3.207317,2.585366,469,51.2414,0.597561,0.402439
2,china,82,47,35,239,193,2156,2187,255.9170,220.9786,0.560976,41,2.914634,2.353659,-31,34.9384,0.573171,0.426829
7,iceland,82,46,36,238,209,2074,2297,230.0624,248.6737,0.353659,41,2.902439,2.548780,-223,-18.6113,0.560976,0.439024
18,panama,82,46,36,255,213,2235,2174,257.4552,241.3937,0.512195,41,3.109756,2.597561,61,16.0615,0.560976,0.439024
3,ethiopia,82,43,39,267,247,2192,2441,239.4199,262.6881,0.243902,41,3.256098,3.012195,-249,-23.2682,0.524390,0.475610


In [10]:
home_wins = games[games['home_goals'] > games['away_goals']]
home_wins.describe()

,home_goals,away_goals,home_shots,away_shots,home_xg,away_xg,home_assists,away_assists,home_penalties_committed,away_penalties_committed,home_penalty_minutes,away_penalty_minutes,went_ot,home_score,away_score,goal_diff,total_goals,shot_diff,total_shots
count,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000,740.000000
mean,3.991892,1.797297,28.333784,24.908108,3.274671,2.630231,6.221622,2.808108,5.805405,7.179730,11.832432,14.677027,0.202703,3.991892,1.797297,2.194595,5.789189,3.425676,53.241892
std,1.639314,1.337294,6.596074,6.186299,0.930712,0.777341,2.929403,2.204299,2.707174,2.936656,5.614019,6.137333,0.402285,1.639314,1.337294,1.518483,2.577910,8.595158,9.469965
min,1.000000,0.000000,10.000000,8.000000,1.094200,0.565100,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000,1.000000,-22.000000,30.000000
25%,3.000000,1.000000,24.000000,21.000000,2.651400,2.080475,4.000000,1.000000,4.000000,5.000000,8.000000,10.000000,0.000000,3.000000,1.000000,1.000000,4.000000,-2.000000,47.000000
50%,4.000000,2.000000,28.000000,24.000000,3.182250,2.549250,6.000000,2.000000,6.000000,7.000000,12.000000,14.000000,0.000000,4.000000,2.000000,2.000000,5.000000,3.000000,53.000000
75%,5.000000,3.000000,32.000000,29.000000,3.818400,3.099200,8.000000,4.000000,7.000000,9.000000,16.000000,18.000000,0.000000,5.000000,3.000000,3.000000,7.000000,9.000000,59.000000
max,11.000000,7.000000,51.000000,48.000000,7.347200,5.888500,19.000000,11.000000,16.000000,19.000000,32.000000,38.000000,1.000000,11.000000,7.000000,8.000000,15.000000,34.000000,95.000000


In [ ]:
'''
# TODO: Calculate 'Home Multiplier' by comparing Home xG/60 vs Away xG/60 across the league.
# TODO: Apply a 'Neutrality Filter' to penalize home-heavy schedules in the rankings.

# --- LOGIC FOR went_ot (The Volatility Filter) ---
# 1. Regulation Performance: Use this to isolate 'Regulation Goal Differential'.
#    A team winning 5-0 in regulation is significantly stronger than a team
#    winning 1-0 in OT. The former shows dominance; the latter shows a coin-flip.
#
# 2. 'The Paper Tiger' Check: Identify teams with high standings but high OT win rates.
#    If a team relies on OT/Shootouts, their Power Ranking should be ADJUSTED DOWN
#    as OT results are less repeatable than 5-on-5 play.
#
# 3. 'The Resilience' Factor: Boost teams with high OT Loss counts.
#    In the standings, they look like losers (0 wins), but in reality,
#    they are competitive enough to hold elite teams to a draw for 60 minutes.
#
# 4. Usage Normalization: Since OT adds extra 'toi', always use 'per 60 minutes'
#    rates (e.g., xG/60) to ensure OT minutes don't artificially inflate total stats.

# --- LOGIC FOR home_off_line (The Roster Strength Factor) ---
# 1. Roster Depth: Compare 'first_off' vs 'second_off' xG/60.
#    - 'One-Line Wonders': Teams with a huge drop-off in quality (e.g., 1st line 3.0 xG, 2nd line 0.5 xG).
#    - 'Balanced Giants': Teams where both lines produce consistently.
#    ACTION: Reward 'Balanced' teams with a higher stability score in rankings.
#
# 2. Situational Power: Isolate 'PP_up' (Power Play) records.
#    - Standing might be low, but if 'PP_up' xG/60 is top 5, they are a 'Danger Team'.
#    ACTION: Add a 'Special Teams Grade' to the final Power Ranking.
#
# 3. 5-on-5 Purity: Filter for 'first_off' and 'second_off' only to find 'Even-Strength' dominance.
#    - This is the most repeatable part of hockey.
#    ACTION: Use Even-Strength xG Differential as 50% of the total Power Ranking weight.
#
# 4. Tactical Matchups: Link with 'away_def_pairing' to see which lines 'crush' weaker defenders.
#    - Identify teams that successfully hunt mismatches (e.g., first_off vs. opponent's second_def).

# --- LOGIC FOR home_def_pairing (The Shutdown Metric) ---
# 1. Shutdown Quality: Calculate 'xG Allowed per 60' for each pairing.
#    A team's 'Defensive Rank' should be heavily weighted by the first_def unit.
#
# 2. Defensive Depth: Measure the 'Reliability Gap' between 1st and 2nd pairs.
#    Teams with a strong second_def are 'Tournament Hardened' and harder to exploit.
#
# 3. PK Specialist Rank: Filter for 'PP_kill_dwn'.
#    Identify teams that effectively suppress xG even when man-down.
#    High PK efficiency is a major signal for 'Playoff Ready' power rankings.
#
# 4. Goal-Save Delta: Compare 'Actual Goals Allowed' vs 'xG Allowed' per pairing.
#    If a pairing allows high xG but zero goals, the goalie is 'bailing them out'.




'''

In [11]:
home_ot = games_unique[['home_team', 'went_ot']].rename(columns={'home_team': 'team'})
away_ot = games_unique[['away_team', 'went_ot']].rename(columns={'away_team': 'team'})

# Combine both lists and sum the OT occurrences
team_ot_counts = pd.concat([home_ot, away_ot]).groupby('team')['went_ot'].sum().reset_index()
team_ot_counts.columns = ['team', 'ot_games_count']

# Sort by the number of OT games
team_ot_counts = team_ot_counts.sort_values(by='ot_games_count', ascending=False)
print(team_ot_counts)

            team  ot_games_count
28           uae              28
20   philippines              26
31       vietnam              25
11        mexico              22
19          peru              22
3       ethiopia              21
13       morocco              21
29            uk              20
24     singapore              20
30           usa              20
18        panama              19
4         france              19
6      guatemala              19
26   switzerland              19
2          china              18
12      mongolia              18
1         canada              18
21        rwanda              18
22  saudi_arabia              18
14   netherlands              18
7        iceland              17
17      pakistan              16
25   south_korea              15
9      indonesia              15
10    kazakhstan              14
16          oman              14
0         brazil              13
8          india              13
27      thailand              13
15   new_z

In [ ]:
# --- LOGIC FOR AWAY COLUMNS (Road Resilience & System Strength) ---
# 1. Road Resilience Score: Aggregate team xG when playing as 'away_team'.
#    - Compare 'Away xG/60' vs 'Home xG/60'.
#    - ACTION: Teams with the smallest "Home-Road Gap" get a Reliability Bonus.
#      They are 'System-Strong' and play well regardless of environment.
#
# 2. Defensive Opponent-Adjustment: Use away_def_pairing to 'weight' offensive success.
#    - Scoring against an opponent's 'first_def' is worth more Power Points
#      than scoring against their 'second_def'.
#    - ACTION: Create a 'Difficulty-Adjusted Goal' metric.
#
# 3. The "Last Change" Penalty: On the road, teams cannot control line matchups.
#    - If a team's 'away_off_line' (1st) still dominates while being 'hunted'
#      by the home coach, they are a Top-Tier Juggernaut.
#
# 4. Data Normalization: Combine Home and Away stats into a single 'Neutral Table'.
#    - This ensures a team's Power Ranking is based on their WHOLE season,
#      not just a favorable home schedule.